# Get Books Recommendations Based on Book Ratings by other users
## Load booksratings data

In [1]:
val rawdata = sc.textFile("/docker/datasets/Book-Crossing/BX-Book-Ratings-clean.csv")

In [2]:
import org.apache.spark.mllib.recommendation._

val ratings = rawdata.map(_.split(",")).flatMap {
    case Array(userId, bookId, rating) => List(Rating(userId.##, bookId.##, rating.toDouble))
    case _ => Nil
}.cache()

## Train Alternate Least Square Model

In [3]:
val rank = 10
val numIterations = 10
val model = ALS.train(ratings, rank, numIterations, 0.01)

## Evaluation

In [6]:
val usersBooks = ratings.map {
    case Rating(user, book, rate) => (user, book)
}

val predictions = model.predict(usersBooks).map {
    case Rating(user, book, rate) => ((user, book), rate)
}

val ratesAndPreds = ratings.map {
    case Rating(user, book, rate) => ((user, book), rate)
}

In [7]:
import scala.math.pow
val MSE = (predictions join ratesAndPreds).values.map {
    case (r1, r2) => pow(r1 - r2, 2)
}.mean

In [8]:
MSE

1.543133789501017

## Recomend 10 Books for some user using SparkSQL and save to Cassandra

In [13]:
sqlContext.sql("SELECT product FROM RECOMENDATIONS WHERE user = 1454240742 ORDER BY rating DESC").show

+-----------+
|    product|
+-----------+
| 1492767647|
| 1189115084|
| -573959997|
| 1064138883|
|-1974867320|
|  567161828|
|-1632106272|
|-1413874720|
| -513295492|
|-1517405952|
| 1871641768|
| 2012697882|
| -706641786|
|  624131577|
| -231294235|
|-1885107065|
|-1067239038|
| 1190855202|
|  869624709|
| -747635905|
+-----------+



# Get Recommentations by user

# Recomend 10 Books for some user using DataFrames

In [8]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext._
import sqlContext.implicits._

val df = model.predict(usersBooks).toDF

In [9]:
df.show()

+----------+-----------+--------------------+
|      user|    product|              rating|
+----------+-----------+--------------------+
|1454240742|-1632106272|    8.07874748460219|
|1481950097|-1283513920|   3.998921658529917|
|  52455867|-1283513920|   8.118608673296709|
|1480216506| -738125816|  10.142597387607847|
|  48764605| -738125816|  10.232640840531621|
|1451555651|-1991032848|                 0.0|
|1453347846|  924811832|                 0.0|
|1481170720| -306767624|   0.648859710056537|
|  50491673| -306767624|  1.8322300157627662|
|  52516593| -306767624|  1.2722275412129598|
|1456236587| -306767624|   6.749916587500888|
|1453257636| -306767624| -0.6042261857159872|
|1479327556| -306767624| 0.07799550700979596|
|  50665645| -306767624|   4.891981331691452|
|1450725253| -306767624|-0.00148863673925...|
|  50577326| -306767624|   7.501813695557477|
|1453320942| -306767624|   5.999499735821199|
|1448843639| -306767624| 0.18292499069853196|
|1481116067|  -44503720|          

In [10]:
df.filter(df("user") == 1454240742)

Name: Compile Error
Message: <console>:41: error: overloaded method value filter with alternatives:
  (conditionExpr: String)org.apache.spark.sql.DataFrame <and>
  (condition: org.apache.spark.sql.Column)org.apache.spark.sql.DataFrame
 cannot be applied to (Boolean)
              df.filter(df("user") == 1454240742)
                 ^
StackTrace: 

In [12]:
df.registerTempTable("RECOMENDATIONS")
sqlContext.sql("SELECT product FROM RECOMENDATIONS WHERE user = 1454240742 ORDER BY rating DESC").show
//     map(_.getInt(0)).take(10)

+-----------+
|    product|
+-----------+
| 1492767647|
| 1189115084|
| -573959997|
| 1064138883|
|-1974867320|
|  567161828|
|-1632106272|
|-1413874720|
| -513295492|
|-1517405952|
| 1871641768|
| 2012697882|
| -706641786|
|  624131577|
| -231294235|
|-1885107065|
|-1067239038|
| 1190855202|
|  869624709|
| -747635905|
+-----------+



Name: Syntax Error.
Message: 
StackTrace: 